<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Libraires And Utilities</h1>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
pyo.init_notebook_mode()
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('vader_lexicon')
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR,AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import random
import pymc3 as pm
import theano.tensor as T
from wordcloud import STOPWORDS
import warnings
import os
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

plt.rc('figure',figsize=(17,13))
plt.rc('figure',autolayout=True)
plt.style.use('ggplot')
#sns.set_style('whitegrid')
sns.set_context('paper',font_scale=2)
def set_seed(seed=31415):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

In [ ]:
t_data = pd.read_csv('/kaggle/input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv',index_col=0)
t_data.date = pd.to_datetime(t_data.date)
t_data.head(3)

<a id="2"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Text Preprocessing</h1>


In [ ]:
text_dict = dict()
for (tweet,date) in zip(t_data.tweet,t_data.date):
    text_dict[(tweet,date)] = text_dict.get((tweet,date),0)+1
    
text_dict = list(text_dict.keys())
f_data = pd.DataFrame(text_dict,columns=['text','date'])

f_data.text =f_data.text.str.lower()

#Remove twitter handlers
f_data.text = f_data.text.apply(lambda x:re.sub('@[^\s]+','',x))

#remove hashtags
f_data.text = f_data.text.apply(lambda x:re.sub(r'\B#\S+','',x))


# Remove URLS
f_data.text = f_data.text.apply(lambda x:re.sub(r"http\S+", "", x))


#remove all single characters
f_data.text = f_data.text.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

# Substituting multiple spaces with single space
f_data.text = f_data.text.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))


# Remove all the special characters
f_data.text = f_data.text.apply(lambda x:' '.join(re.findall(r'\w+', x)))





<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Vader Sentiment Analysis</h1>


In [ ]:
#Vader Setiment Analysis
sid = SIA()
f_data['sentiments']           = f_data['text'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
f_data['Positive Sentiment']   = f_data['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
f_data['Neutral Sentiment']    = f_data['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
f_data['Negative Sentiment']   = f_data['sentiments'].apply(lambda x: x['neg']+1*(10**-6))
f_data.drop(columns=['sentiments'],inplace=True)


<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Naive Feature Engineering</h1>


In [ ]:


#Number of Words
f_data['Number_Of_Words'] = f_data.text.apply(lambda x:len(x.split(' ')))
#Average Word Length
f_data['Mean_Word_Length'] = f_data.text.apply(lambda x:np.round(np.mean([len(w) for w in x.split(' ')]),2) )

<a id="4"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Exploratory Data Analysis</h1>


In [ ]:
plt.subplot(2,1,1)
plt.title('Distriubtion Of Sentiments Across Our Tweets',fontsize=19,fontweight='bold')
sns.kdeplot(f_data['Negative Sentiment'],label='Negative Sentiment',lw=2.5)
sns.kdeplot(f_data['Positive Sentiment'],label='Positive Sentiment',lw=2.5)
sns.kdeplot(f_data['Neutral Sentiment'], label='Neutral Sentiment',lw=2.5 )
plt.legend()
plt.subplot(2,1,2)
plt.title('CDF Of Sentiments Across Our Tweets',fontsize=19,fontweight='bold')
sns.kdeplot(f_data['Negative Sentiment'],cumulative=True ,label='Negative Sentiment',lw=2.5)
sns.kdeplot(f_data['Positive Sentiment'],cumulative=True ,label='Positive Sentiment',lw=2.5)
sns.kdeplot(f_data['Neutral Sentiment'],cumulative=True  ,label='Neutral Sentiment' ,lw=2.5)
plt.xlabel('Sentiment Value',fontsize=19)
plt.legend()
plt.show()

**Observation**: Notice that all our sentiments follow a bimodal distribution, suggesting that there might be two underlying groups in our dataset; we will indeed explore and try to understand the two modes better later in our analysis.

<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Time Based Analysis</h1>


In [ ]:
#Sorting And Feature Engineering
f_data = f_data.sort_values(by='date')
ft_data=f_data.copy()
ft_data['date'] = pd.to_datetime(f_data['date']).dt.date

ft_data['year']         = pd.DatetimeIndex(ft_data['date']).year
ft_data['month']        = pd.DatetimeIndex(ft_data['date']).month
ft_data['day']          = pd.DatetimeIndex(ft_data['date']).day
ft_data['day_of_year']  = pd.DatetimeIndex(ft_data['date']).dayofyear
ft_data['quarter']      = pd.DatetimeIndex(ft_data['date']).quarter
ft_data['season']       = ft_data.month%12 // 3 + 1

In [ ]:
f_data=f_data.reset_index().drop(columns=['index'])
partitions = []
partitions.append(f_data.loc[44:np.round(len(f_data)/3,0)-1,:])
partitions.append(f_data.loc[np.round(len(f_data)/3,0):2*int(len(f_data)/3)-1,:])
partitions.append(f_data.loc[2*np.round(len(f_data)/3,0):3*int(len(f_data)/3)-1,:])



neg_part_means =[]
neg_part_std   =[]
pos_part_means =[]
pos_part_std   =[]
for part in partitions:
    neg_part_means.append(part['Negative Sentiment'].mean())
    neg_part_std.append(part['Negative Sentiment'].std())
    pos_part_means.append(part['Positive Sentiment'].mean())
    pos_part_std.append(part['Positive Sentiment'].std())
    
res_df = pd.DataFrame({'Positive Sentiment Mean':pos_part_means,'Negative Sentiment Mean':neg_part_means,'Positive Sentiment SD':pos_part_std,'Negative Sentiment SD':neg_part_std},
                     index = [f'Partition_{i}' for i in range(1,4)])


def highlight_greater(x):
    temp = x.copy()
    temp = temp.round(0).astype(int)
    m1 = (temp['Partition_1_Mean'] == temp['Partition_2_Mean'])
    m2 = (temp['Partition_1_SD'] == temp['Partition_2_SD'])
    m3 = (temp['Partition_1_Mean'] < temp['Partition_2_Mean']+3) & (temp['Partition_1_Mean'] > temp['Partition_2_Mean']-3)
    m4 = (temp['Partition_1_SD'] < temp['Partition_2_SD']+3) & (temp['Partition_1_SD'] > temp['Partition_2_SD']-3)

    df1 = pd.DataFrame('background-color: ', index=x.index, columns=x.columns)
    #rewrite values by boolean masks
    df1['Partition_1_Mean'] = np.where(~m1, 'background-color: {}'.format('salmon'),        df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(~m1, 'background-color: {}'.format('salmon'),        df1['Partition_2_Mean'])
    df1['Partition_1_Mean'] = np.where(m3, 'background-color: {}'.format('gold'),           df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(m3, 'background-color: {}'.format('gold'),           df1['Partition_2_Mean'])
    df1['Partition_1_Mean'] = np.where(m1, 'background-color: {}'.format('mediumseagreen'), df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(m1, 'background-color: {}'.format('mediumseagreen'), df1['Partition_2_Mean'])

    df1['Partition_1_SD'] = np.where(~m2, 'background-color: {}'.format('salmon'),        df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(~m2, 'background-color: {}'.format('salmon'),        df1['Partition_2_SD'])
    df1['Partition_1_SD'] = np.where(m4, 'background-color: {}'.format('gold'),           df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(m4, 'background-color: {}'.format('gold'),           df1['Partition_2_SD'])
    df1['Partition_1_SD'] = np.where(m2, 'background-color: {}'.format('mediumseagreen'), df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(m2, 'background-color: {}'.format('mediumseagreen'), df1['Partition_2_SD'])

    return df1



#res_df.style.apply(highlight_greater,axis=None)
res_df = res_df.T
res_df = pd.DataFrame(res_df.values,columns=res_df.columns,index=['Positive Sentiment','Negative Sentiment','Positive Sentiment','Negative Sentiment'])
res_df = pd.concat([res_df.iloc[:2,:],res_df.iloc[2:,:]],axis=1)
res_df.columns = ['Partition_1_Mean','Partition_2_Mean','Partition_3_Mean','Partition_1_SD','Partition_2_SD','Partition_3_SD']
res_df.style.apply(highlight_greater,axis=None)

**Observation**: It seems that the mean and standard deviation are constant in our partitions which may suggest the sentiments are stationary over time but still, it is not enough evidence; we will keep testing the stationarity.

In [ ]:
fig = make_subplots(rows=3, cols=2)

for idx,prt in enumerate(partitions):
    fig.add_trace(
    go.Scatter(x=prt['date'], y=prt['Positive Sentiment'],name=f'Positive Part {idx+1}'),
    row=idx+1, col=1)
    fig.add_trace(
    go.Scatter(x=prt['date'], y=prt['Negative Sentiment'],name=f'Negative Part {idx+1}'),
    row=idx+1, col=2)

fig.update_layout(height=600, width=900, title_text="Distibution Of Daily Sentiments Over Our Time Line For Each Partition")
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=2, subplot_titles=('Observed Pos', 'Observed Neg', 'Trend Pos','Trend Neg','Seasonal Pos','Seasonal Neg','Residual Pos','Residual Neg'))
scope = 'month'
b_date_mean = ft_data.groupby(by=scope).mean().reset_index()


lbl = ['Positive','Negative']

for idx,column in enumerate(['Positive Sentiment','Negative Sentiment']):
    res = seasonal_decompose(b_date_mean[column], period=3, model='additive', extrapolate_trend='freq')
    
    fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=res.observed,name='{} Observed'.format(lbl[idx])),
    row=1, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=res.trend,name='{} Trend'.format(lbl[idx])),
    row=2, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=res.seasonal,name='{} Seasonal'.format(lbl[idx])),
    row=3, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=res.resid,name='{} Residual'.format(lbl[idx])),
    row=4, col=idx+1)
            
fig.update_layout(height=600, width=900, title_text="Decomposition Of Our Monthly Average Sentiments into Trend,Level,Seasonality and Residuals")
fig.show()

**Observation**: when looking at the overall average monthly sentiment in Trump's tweets, we see that the negative sentiment tends to drops towards December and the positive sentiment tends to increase around December on average.

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 10))

ax[0].set_title('Positive Autocorrelation Analysis ',fontsize=18,fontweight='bold')
autocorrelation_plot(b_date_mean['Positive Sentiment'],ax=ax[0],lw=3)
ax[1].set_title('Negative Autocorrelation Analysis ',fontsize=18,fontweight='bold')
autocorrelation_plot(b_date_mean['Negative Sentiment'],ax=ax[1],color='tab:red',lw=3)
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 10))
ax[0].set_ylim(-1.1,1.1)
ax[1].set_ylim(-1.1,1.1)

plot_pacf(b_date_mean['Negative Sentiment'],lags=5, ax=ax[0],title='Partial Autocorrelation Negative')
plot_pacf(b_date_mean['Positive Sentiment'],lags=5, ax=ax[1],color='tab:blue',title='Partial Autocorrelation Positive')
plt.show()

**Observation**: Looking at our ACF and PACF plots, we see no significant direct correlations between the data and lagged version of itself except the indirect correlation observed in the PACF with the fifth lag.
Being able to predict a month's sentiment based on the previous months may indicate a seasonal behavior among Trump's tweets.


<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Modeling Future Negative Sentiment</h1>



In [ ]:
arma_5 = SARIMAX(endog=b_date_mean['Negative Sentiment'],order=(0,0,5)).fit()


In [ ]:
fig = plt.figure(figsize=(16,9))
fig = arma_5.plot_diagnostics(fig=fig, lags=5)

In [ ]:
predicted_AR_1 = arma_5.predict(1)

output = pd.DataFrame({'Prediction':predicted_AR_1,'Actual':b_date_mean['Positive Sentiment']})

fig = make_subplots(
    rows=3, cols=2,subplot_titles=('','Actual','Predictions','Residuals'),
    vertical_spacing=0.09,
    specs=[[{"type": "table","rowspan": 3}     ,{"type": "scatter"}] ,
           [None                               ,{"type": "scatter"}]            ,           
           [None                               ,{"type": "scatter"}]                           
          ]
)

fig.add_trace(
    go.Scatter(
        x=b_date_mean[scope],
        y=output["Actual"],
        mode="lines+markers",
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=b_date_mean[scope],
        y=output["Prediction"],
        mode="lines+markers",
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Prediction"])),
        y=output["Prediction"]-output["Actual"],
        mode="lines+markers",
    ),
    row=3, col=2
)

fig.add_trace(
    go.Table(
        header=dict(
            values=['Prediction','Actual'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[output[k].tolist() for k in output.columns],
            align = "left")
    ),
    row=1, col=1
)



fig.add_shape(type="line",
    x0=0, y0=(output["Prediction"]-output["Actual"]).mean(), x1=len(output["Prediction"]), y1=(output["Prediction"]-output["Actual"]).mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x3', 
        yref='y3'
)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Prediction Evaluation",
)

fig.show()

**Observation**:  We see that naively using an MA(5) model to try and predict the seasonal behavior we hypothesized earlier came out to be far from perfect; if it would be beneficial to come to such insight, it might be better to try and generate a more sophisticated ARIMA or GARCH model.


In [ ]:
b_date_mean = ft_data.groupby(by=scope).mean().reset_index()
b_date_std = ft_data.groupby(by=scope).std().reset_index()


fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=(f'{scope}ly Average Positive Sentiment',  f'{scope}ly Average Negative Sentiment'))

fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=b_date_mean['Positive Sentiment'],name='Positive Sentiment Mean'),
    row=1, col=1
)

    
#positive mean
fig.add_shape(type="line",
    x0=b_date_mean[scope].values[0], y0=b_date_mean['Positive Sentiment'].mean(), x1=b_date_mean[scope].values[-1], y1=b_date_mean['Positive Sentiment'].mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean'
)

fig.add_annotation(x=b_date_mean[scope].values[3], y=b_date_mean['Positive Sentiment'].mean(),
            text=r"$\mu : {:.2f}$".format(b_date_mean['Positive Sentiment'].mean()),
            showarrow=True,
            arrowhead=3,
            yshift=10)



fig.add_trace(
    go.Scatter(x=b_date_mean[scope], y=b_date_mean['Negative Sentiment'],name='Negative Sentiment Mean'),
    row=2, col=1
)

#negative mean
fig.add_shape(type="line",
    x0=b_date_mean[scope].values[0], y0=b_date_mean['Negative Sentiment'].mean(), x1=b_date_mean[scope].values[-1], y1=b_date_mean['Negative Sentiment'].mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x2', 
        yref='y2'
)

# fig.add_annotation(x=b_date_mean[scope].values[3], y=b_date_mean['Negative Sentiment'].mean(),
#             text=r"$\mu : {:.2f}$".format(b_date_mean['Negative Sentiment'].mean()),
#             showarrow=True,
#             arrowhead=3,
#             yshift=10,
#             xref='x2', 
#             yref='y2')



# fig.add_annotation(x=b_date_mean[scope].values[5], y=b_date_mean['Negative Sentiment'].mean()+0.01,
#             text=r"Start Of Decline",
#             showarrow=True,
#             arrowhead=6,
#             yshift=10,
#             xref='x2', 
#             yref='y2')

# fig.add_annotation(x=b_date_mean[scope].values[15], y=.024,
#             text=r"Start Of Incline",
#             showarrow=True,
#             arrowhead=6,
#             yshift=10,
#             xref='x2', 
#             yref='y2')

fig['layout']['xaxis2']['title'] = scope
fig.update_layout(height=700, width=900, title_text="Sentiment Average Change With Time")
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=(f'{scope}ly Deviation in Positive Sentiment',  f'{scope}ly Deviation in Negative Sentiment'))

fig.add_trace(
    go.Scatter(x=b_date_std[scope], y=b_date_std['Positive Sentiment'],name='Positive Sentiment SD'),
    row=1, col=1
)

#negative mean
fig.add_shape(type="line",
    x0=b_date_std[scope].values[0], y0=b_date_std['Negative Sentiment'].mean(), x1=b_date_std[scope].values[-1], y1=b_date_std['Negative Sentiment'].mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x2', 
        yref='y2'
)

#negative mean
fig.add_shape(type="line",
    x0=b_date_std[scope].values[0], y0=b_date_std['Positive Sentiment'].mean(), x1=b_date_std[scope].values[-1], y1=b_date_std['Positive Sentiment'].mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y1'
)


fig.add_trace(
    go.Scatter(x=b_date_std[scope], y=b_date_std['Negative Sentiment'],name='Negative Sentiment SD'),
    row=2, col=1
)

fig['layout']['xaxis2']['title'] = scope
fig.update_layout(height=700, width=900, title_text="Sentiment Deviation Change With Time")
fig.show()

In [ ]:
b_date_count = ft_data.groupby(by=scope).count().reset_index()
b_date_count = b_date_count.rename(columns={ 'date':f'Tweets Per {scope}'})
fig = ex.line(b_date_count,x=scope,y=f'Tweets Per {scope}')

# fig.add_annotation(x=b_date_mean['date'].values[15], y=.024,
#             text=r"Start Of Incline",
#             showarrow=True,
#             arrowhead=6,
#             yshift=10)


fig.add_shape(type="line",
    x0=b_date_count[scope].values[0], y0=b_date_count['Negative Sentiment'].mean(), x1=b_date_count[scope].values[-1], y1=b_date_count['Negative Sentiment'].mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
)

fig.update_traces(mode="markers+lines")
fig.update_layout(hovermode="x unified")


fig.update_layout(title=f'<b>{scope}ly Tweet Count<b>')
fig.show()

**Observation**:  It appears that there are significantly more tweets with insults in June and October; we will investigate those months to try and answer the question, what happened in those months which caused the increased amount of tweets.


In [ ]:
june = ft_data.query('month==7')
october = ft_data.query('month==10')

october_text = ' '.join(october.text)
june_text    = ' '.join(june.text)
october_text = ' '.join([i for i in october_text.split(' ') if len(i) > 2])
june_text    = ' '.join([i for i in june_text.split(' ') if len(i) > 2])

pwc = WordCloud(width=600,height=400,collocations = False,background_color='white').generate(october_text)
nwc = WordCloud(width=600,height=400,collocations = False,background_color='white').generate(june_text)

plt.subplot(1,2,1)
plt.title('Common Words Among October Tweets',fontsize=16,fontweight='bold')
plt.imshow(pwc)
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Common Words Among June Tweets',fontsize=16,fontweight='bold')
plt.imshow(nwc)
plt.axis('off')

plt.show()

**Observation**:  It seems that the peaks may be due to Trump's political campaigns; we can see that the October word-cloud contains "Biden" as one of the most common words; similarly, the word-cloud of June contains "Hillary."

In [ ]:
ag_june = nltk.FreqDist(june_text.split(' '))
ag_october = nltk.FreqDist(october_text.split(' '))


token=nltk.word_tokenize(' '.join(i for i in june_text.split(' ') if i not in STOPWORDS and i.isalpha()))
june_bigram=ngrams(token,2)
june_trigram=ngrams(token,3)

token=nltk.word_tokenize(' '.join(i for i in october_text.split(' ') if i not in STOPWORDS and i.isalpha()))
october_bigram=ngrams(token,2)
october_trigram=ngrams(token,3)


june_bdict = dict()
for i in list(june_bigram):
    june_bdict[i] = june_bdict.get(i,0)+1
    
june_bdict = {k: v for k, v in sorted(june_bdict.items(), key=lambda item: item[1],reverse=True)}
    
october_bdict = dict()
for i in list(october_bigram):
    october_bdict[i] = october_bdict.get(i,0)+1
october_bdict = {k: v for k, v in sorted(october_bdict.items(), key=lambda item: item[1],reverse=True)}


june_tridict = dict()
for i in list(june_trigram):
    june_tridict[i] = june_tridict.get(i,0)+1
    
june_tridict = {k: v for k, v in sorted(june_tridict.items(), key=lambda item: item[1],reverse=True)}
    
october_tridict = dict()
for i in list(october_trigram):
    october_tridict[i] = october_tridict.get(i,0)+1
october_tridict = {k: v for k, v in sorted(october_tridict.items(), key=lambda item: item[1],reverse=True)}


def calculate_tri_prob(tri,firt_order,second_order,third_order):
    fw,sw,tw = tri
    p_fw=firt_order[fw]
    p_fwsw = second_order[(fw,sw)]/p_fw
    p_fwswtw = third_order[(fw,sw,tw)]/p_fw
    return p_fwswtw

def calculate_sentence_prob(sentence,firt_order,second_order):
    p_first = firt_order[sentence[0]]/len(firt_order.keys())
    p_second_first = second_order[(sentence[0],sentence[1])]/len(second_order)
    p_thid_second = second_order[(sentence[1],sentence[2])]/len(second_order)
    return p_first*p_second_first*p_thid_second

top_10_tri_june = []

for key in list(june_tridict.keys())[:15]:
    top_10_tri_june.append((key,calculate_tri_prob(key,ag_june,june_bdict,june_tridict)))
    
    
june_tri_df = pd.DataFrame(top_10_tri_june,columns=['Sentence','Probability Given First Words'])

june_tri_df['First Word'] = june_tri_df.Sentence.apply(lambda x: x[0])
june_tri_df['Sentence Probability-Markov Assumption'] = june_tri_df.Sentence.apply(lambda x: calculate_sentence_prob(x,ag_june,june_bdict))
june_tri_df = june_tri_df[['First Word','Sentence','Probability Given First Words','Sentence Probability-Markov Assumption']]
june_tri_df.Sentence=june_tri_df.Sentence.apply(lambda x: ' '.join(x))
june_tri_df.style.background_gradient(cmap='coolwarm')


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Above is a trigram of 15 sentences that start with one of the top 10 words  in June  and the probability that the sentence will appear in Trump's Tweets given he starts the setntce with the one the top 10 first words.</span></p>
<p><br></p>

In [ ]:
top_10_tri_october = []

for key in list(october_tridict.keys())[:15]:
    top_10_tri_october.append((key,calculate_tri_prob(key,ag_october,october_bdict,october_tridict)))
    
    
october_tri_df = pd.DataFrame(top_10_tri_october,columns=['Sentence','Probability Given First Words'])

october_tri_df['First Word'] = october_tri_df.Sentence.apply(lambda x: x[0])
october_tri_df               = october_tri_df[['First Word','Sentence','Probability Given First Words']]
october_tri_df.Sentence=october_tri_df.Sentence.apply(lambda x: ' '.join(x))
october_tri_df.style.background_gradient(cmap='coolwarm')


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Above is a trigram of 15 sentences that start with one of the top 10 words  in October  and the probability that the sentence will appear in Trump's Tweets given he starts the setntce with the one the top 10 first words.</span></p>
<p><br></p>

In [ ]:
plt.subplot(2,1,1)
ax = sns.barplot(x=[' '.join(i) for i in list(june_bdict.keys())[:10]],y=list(june_bdict.values())[:10])
ax.set_title('Most Common Two Word Sentences In June')
ax.set_xlabel('Sentence')
ax.set_ylabel('Frequency')
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.subplot(2,1,2)
ax = sns.barplot(x=[' '.join(i) for i in list(october_bdict.keys())[:10]],y=list(october_bdict.values())[:10])
ax.set_title('Most Common Two Word Sentences In October')
ax.set_xlabel('Sentence')
ax.set_ylabel('Frequency')
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.show()

<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Probabilistic Inference</h1>



In [ ]:
plt.title('Distribution Of Positive Sentimnet',fontweight='bold',fontsize=20)
ax= sns.kdeplot(f_data['Positive Sentiment'],label='Positive Sentiment',lw=3)
ax= sns.distplot(f_data['Positive Sentiment'],label='Positive Sentiment')

plt.legend(prop=dict(size=23))
plt.show()

**Observation**:  As we saw earlier, our sentiments are bimodal, and it is of interest to understand what generates the bimodality and what is unique about each underlying group.

<p>In order to get a better understanding, we will make the assumption that our data consists of two clusters; we will assume that each point (Positive sentiment of a tweet) belongs to either cluster 1 or cluster 2 with probability $P_1$ and $1-P_1$ respectfully.
Our goal will be to create a posterior space where the centers or our distribution modes may reside together with the standard deviations.
We will define $P_1$ to be a random uniform variable and try and use MCMC to converge on potential $P_1$ value while exploring the possible centers ($\mu_i$) and SD's ($\sigma_i$). <p>


In [ ]:

with pm.Model():
    #Probability Of Belonging To Cluster 1
    p1 = pm.Uniform('p',0,1)
    #Probability Of Belonfing To Cluster 2
    p2 = 1-p1
    p = T.stack([p1, p2])
    assignment = pm.Categorical("assignment", p, 
                                shape=f_data.shape[0],
                                testval=np.random.randint(0, 2, f_data.shape[0]))
    
    sds = pm.Uniform('sds',0,1,shape=2)
    centers = pm.Normal("centers", 
                        mu=np.array([0, 0.1]), 
                        sd=np.array([0.3, 0.3]), 
                        shape=2)
    center_i = pm.Deterministic('center_i', centers[assignment])
    sd_i = pm.Deterministic('sd_i', sds[assignment])
    observations = pm.Normal("obs", mu=center_i, sd=sd_i, observed=f_data['Positive Sentiment'])
    
    step1 = pm.Metropolis(vars=[p, sds, centers])
    step2 = pm.ElemwiseCategorical(vars=[assignment])
    trace = pm.sample(5000, step=[step1, step2])

In [ ]:
std_trace = trace["sds"][8000:]
center_trace = trace["centers"][8000:]

colors = ["#348ABD", "#A60628"] if center_trace[-1, 0] > center_trace[-1, 1] \
    else ["#A60628", "#348ABD"]
_i = [1, 2, 3, 4]
for i in range(2):
    plt.subplot(2, 2, _i[2 * i])
    plt.title("Posterior of center of cluster %d" % i)
    plt.hist(center_trace[:, i], color=colors[i],
             histtype="stepfilled")

    plt.subplot(2, 2, _i[2 * i + 1])
    plt.title("Posterior of standard deviation of cluster %d" % i)
    plt.hist(std_trace[:, i], color=colors[i],
             histtype="stepfilled")

**Observation**:  We see that after sampling from our posterior space, thousand of samples which are close to are real unknown parameters that the center of the first cluster is 0.1 with high confidence and the center of the second cluster is 0 also with high confidence, as for the standard deviation we have a lower confidence interval, but we can see that most likely the standard deviation of the first cluster is between 0.082 and 0.086 , as for the second cluster the standard deviation is between w.25 * 1e-10 and 2.45*1e-10 

In [ ]:
import matplotlib as mpl
cmap = mpl.colors.LinearSegmentedColormap.from_list("BMH", colors)
assign_trace = trace["assignment"]
plt.subplot(2,1,1)

plt.scatter(f_data['Positive Sentiment'], 1 - assign_trace.mean(axis=0), cmap=cmap,
        c=assign_trace.mean(axis=0), s=50)
plt.ylim(-0.05, 1.05)
#plt.xlim(35, 300)
plt.title("Probability of data point belonging to cluster 0")
plt.ylabel("probability")
plt.xlabel("value of data point");

plt.subplot(2,1,2)
plt.scatter(f_data['Positive Sentiment'], 1 - assign_trace.mean(axis=0), cmap=cmap,
        c=assign_trace.mean(axis=0), s=50)
plt.ylim(-0.001, 0.003)
#plt.xlim(35, 300)
plt.title("Probability of data point belonging to cluster 1")
plt.ylabel("probability")
plt.xlabel("value of data point");

In [ ]:
import scipy.stats as stats
norm = stats.norm
x = np.linspace(0, 1, 50)
posterior_center_means = center_trace.mean(axis=0)
posterior_std_means = std_trace.mean(axis=0)
posterior_p_mean = trace["p"].mean()
data = f_data['Positive Sentiment']

plt.hist(data, bins=20, histtype="step", density=True, color="k",
     lw=2, label="histogram of data")
y = posterior_p_mean * norm.pdf(x, loc=posterior_center_means[0],
                                scale=posterior_std_means[0])
plt.plot(x, y, label="Cluster 0 (using posterior-mean parameters)", lw=3)
plt.fill_between(x, y, color=colors[1], alpha=0.3)

y = (1 - posterior_p_mean) * norm.pdf(x, loc=posterior_center_means[1],
                                      scale=posterior_std_means[1]*10000)
plt.plot(x, (y-y.mean())/y.std(), label="Cluster 1 (using posterior-mean parameters)", lw=3)
plt.fill_between(x, (y-y.mean())/y.std(), color=colors[0], alpha=0.3)

plt.legend(loc="upper left")
plt.title("Visualizing Clusters using posterior-mean parameters");

<a id="3"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Text Decomposition Analysis</h1>



In [ ]:
NUMBER_OF_COMPONENTS = 450

CVZ = CountVectorizer()
SVD = TruncatedSVD(NUMBER_OF_COMPONENTS)

text_data = f_data.text.copy()
text_data = text_data.apply(lambda x: ' '.join([word for word in x.split() if word not in STOPWORDS and len(word) > 1]).strip())

stemmer= PorterStemmer()
lemmatizer=WordNetLemmatizer()

text_data = text_data.apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))
text_data = text_data.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

C_vector = CVZ.fit_transform(text_data)


pc_matrix = SVD.fit_transform(C_vector)

evr = SVD.explained_variance_ratio_
total_var = evr.sum() * 100
cumsum_evr = np.cumsum(evr)

trace1 = {
    "name": "individual explained variance", 
    "type": "bar", 
    'y':evr}
trace2 = {
    "name": "cumulative explained variance", 
    "type": "scatter", 
     'y':cumsum_evr}
data = [trace1, trace2]
layout = {
    "xaxis": {"title": "Principal components"}, 
    "yaxis": {"title": "Explained variance ratio"},
  }
fig = go.Figure(data=data, layout=layout)
fig.update_layout(     title='{:.2f}% of the Tweet Text Variance Can Be Explained Using {} Words'.format(np.sum(evr)*100,NUMBER_OF_COMPONENTS))
fig.show()

In [ ]:
best_fearures = [[CVZ.get_feature_names()[i],SVD.components_[0][i]] for i in SVD.components_[0].argsort()[::-1]]
worddf = pd.DataFrame(np.array(best_fearures[:450])[:,0]).rename(columns={0:'Word'})
worddf['Explained Variance'] =  np.round(evr*100,2)
worddf['Explained Variance'] =worddf['Explained Variance'].apply(lambda x:str(x)+'%')
app = []
for word in worddf.Word:
    total_count = 0
    for tweet in text_data:
        if tweet.find(word)!= -1:
            total_count+=1
    app.append(total_count)
worddf['Appeared_On_X_Tweets'] = app
worddf

fig = go.Figure()
fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Word<b>',"<b>Explaines X% of Variance<b>",'<b>Appeared On X Tweets<b>'],
            font=dict(size=19,family="Lato"),
            align="center"
        ),
        cells=dict(
            values=[worddf[k].tolist() for k in ['Word',"Explained Variance",'Appeared_On_X_Tweets']],
            align = "center")
    ),
    
)

fig.show()